In [12]:
import pandas as pd
from dateutil.relativedelta import relativedelta
from google.colab import files
import io
import re

In [13]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]

df = pd.read_csv(io.BytesIO(uploaded[filename]))
print(df.head())

Saving expanded_tourism_dataset.csv to expanded_tourism_dataset (1).csv
  series_id           start_date    Month1      Month2      Month3  \
0        T1  1979-01-01 00-00-00   1149.87   1053.8002   1388.8798   
1        T2  1979-01-01 00-00-00  65072.37  48612.2000  58452.5900   
2        T3  1985-01-01 00-00-00  37047.00  32836.0000  51094.0000   
3        T4  1985-01-01 00-00-00   7611.00   6859.0000   9247.0000   
4        T5  1985-01-01 00-00-00  27693.00  15075.0000  22602.0000   

       Month4      Month5      Month6       Month7       Month8  ...  \
0   1783.3702   1921.0252   2704.9449    4184.4135    4148.3542  ...   
1  57033.9700  71498.9500  79187.1100  101896.1000  115971.8000  ...   
2  52629.0000  56884.0000  52955.0000   56012.0000   50462.0000  ...   
3  10924.0000   9064.0000   7417.0000   10467.0000    7921.0000  ...   
4  23530.0000  26817.0000  22696.0000   20433.0000   24152.0000  ...   

   Month324  Month325  Month326  Month327  Month328  Month329  Month330  \

In [14]:
value_cols = df.columns[2:]
print(value_cols.tolist())

['Month1', 'Month2', 'Month3', 'Month4', 'Month5', 'Month6', 'Month7', 'Month8', 'Month9', 'Month10', 'Month11', 'Month12', 'Month13', 'Month14', 'Month15', 'Month16', 'Month17', 'Month18', 'Month19', 'Month20', 'Month21', 'Month22', 'Month23', 'Month24', 'Month25', 'Month26', 'Month27', 'Month28', 'Month29', 'Month30', 'Month31', 'Month32', 'Month33', 'Month34', 'Month35', 'Month36', 'Month37', 'Month38', 'Month39', 'Month40', 'Month41', 'Month42', 'Month43', 'Month44', 'Month45', 'Month46', 'Month47', 'Month48', 'Month49', 'Month50', 'Month51', 'Month52', 'Month53', 'Month54', 'Month55', 'Month56', 'Month57', 'Month58', 'Month59', 'Month60', 'Month61', 'Month62', 'Month63', 'Month64', 'Month65', 'Month66', 'Month67', 'Month68', 'Month69', 'Month70', 'Month71', 'Month72', 'Month73', 'Month74', 'Month75', 'Month76', 'Month77', 'Month78', 'Month79', 'Month80', 'Month81', 'Month82', 'Month83', 'Month84', 'Month85', 'Month86', 'Month87', 'Month88', 'Month89', 'Month90', 'Month91', 'Month9

In [15]:
df_long = df.melt(id_vars=['series_id', 'start_date'],
                  value_vars=value_cols,
                  var_name='month_index',
                  value_name='value')

In [16]:
df_long['start_date'] = pd.to_datetime(df_long['start_date'])

<ipython-input-16-2d58871fc42f>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_long['start_date'] = pd.to_datetime(df_long['start_date'])


In [17]:
df_long['month_offset'] = df_long['month_index'].str.extract('(\d+)').astype(int) - 1

In [18]:
df_long['date'] = df_long.apply(lambda row: row['start_date'] + relativedelta(months=row['month_offset']), axis=1)
df_long['date'] = df_long['date'].dt.to_period('M')

<ipython-input-18-b1ae2d40207f>:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_long['date'] = df_long['date'].dt.to_period('M')


In [19]:
df_final = df_long[['series_id', 'date', 'value']]
print(df_final.head(10))

  series_id     date     value
0        T1  1979-01   1149.87
1        T2  1979-01  65072.37
2        T3  1985-01  37047.00
3        T4  1985-01   7611.00
4        T5  1985-01  27693.00
5        T6  1985-01  12570.00
6        T7  1985-01   4289.00
7        T8  1985-01  55563.00
8        T9  1985-01   4262.00
9       T10  1985-01   6699.00


In [20]:
def natural_key(s):
    return tuple(int(text) if text.isdigit() else text.lower() for text in re.split('(\d+)', s))

In [23]:
df_final = df_final.sort_values(
    by=['series_id', 'date'],
    key=lambda col: col.map(natural_key) if col.name == 'series_id' else col
)
print(df_final.head(10))

df_final = df_final.dropna(subset=['value'])

     series_id     date      value
0           T1  1979-01  1149.8700
366         T1  1979-02  1053.8002
732         T1  1979-03  1388.8798
1098        T1  1979-04  1783.3702
1464        T1  1979-05  1921.0252
1830        T1  1979-06  2704.9449
2196        T1  1979-07  4184.4135
2562        T1  1979-08  4148.3542
2928        T1  1979-09  2620.7251
3294        T1  1979-10  1650.3001


In [24]:
output_filename = 'preprocessed_data.csv'
df_final.to_csv(output_filename, index=False)
files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>